In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
datapath = 'E:/Dissertation Data'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

#from glove import Corpus, Glove

from gensim.models import Word2Vec

In [3]:
from nltk.metrics import (
    BigramAssocMeasures,
    TrigramAssocMeasures,
    NgramAssocMeasures,
)

from nltk.metrics.spearman import (
    spearman_correlation,
    ranks_from_scores,
)

We then automatically score the million most strongly associated
n-grams (i.e., roughly the top 5% of the Poisson-ranked
list) for compositionality.

Using word2vec (Mikolov et al., 2013) with the parameters
found to be most effective by Baroni et al. (2014), we
build a word embedding vector for every simplex word in
the vocabulary (ca. 1M types), as well as for each MWE candidate.

* Continuous bag of words model with 400-dimensional vectors, window size 5, subsampling with t = 10^-5, negative sampling with 10 samples. We build vectors only for tokens observed 20 times or more in the corpus.

We then compute the cosine similarity of the vector
representation for a MWE candidate with the vectors of its
constituent words, and take the arithmetic mean. 
In scoring
the compositionality of a candidate, we do not measure the
cosine similarity of the MWE with any stop words it may
contain, as stop words may be assumed to be semantically
uninformative.
* Stop words are taken here to be the 50 most frequent words in the vocabulary.

In [4]:
# Stopwords from corpus - 50 most frequent

with open(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/stop.pkl', 'rb') as pfile:
    stop = pickle.load(pfile)

In [5]:
stop

{'%|NN',
 '0|CD',
 '1|CD',
 '2|CD',
 '3|CD',
 'He|PRP',
 'In|IN',
 'It|PRP',
 'The|DT',
 'also|RB',
 'and|CC',
 'an|DT',
 'are|VBP',
 'as|IN',
 'at|IN',
 'a|DT',
 'be|VB',
 'but|CC',
 'by|IN',
 'first|JJ',
 'for|IN',
 'from|IN',
 'had|VBD',
 'has|VBZ',
 'he|PRP',
 'his|PRP$',
 'in|IN',
 'is|VBZ',
 'its|PRP$',
 'it|PRP',
 'not|RB',
 'of|IN',
 'one|CD',
 'on|IN',
 'or|CC',
 's|POS',
 'that|IN',
 'that|WDT',
 'their|PRP$',
 'the|DT',
 'to|IN',
 'to|TO',
 'two|CD',
 'was|VBD',
 'were|VBD',
 'which|WDT',
 'who|WP',
 'with|IN',
 '|HYPH',
 '–|SYM'}

In [6]:
ngram_eval = pd.read_pickle(datapath+'/Corpora/wiki/enwiki_20200520/Tagged/ngram_eval.pkl')

ngram_eval

,ngram,freq,poisson,len,batch
0,"(311.22|CD, 500.86|CD)",20,-667.084219,2,1
1,"(Makhaya|NNP, Ntini|NNP)",20,-667.084219,2,1
2,"(resting_place_coordinates|NNS, burial_place|VBP)",20,-667.084219,2,1
3,"(Dechawat|NNP, Poomjaeng|NNP)",20,-667.084219,2,1
4,"(MSC1|NN, MSC2|NN)",20,-667.084219,2,1
...,...,...,...,...,...
499995,"(US|NNP, Steel|NNP)",32,-1679.033919,2,3
499996,"(Harry|NNP, Wilson|NNP)",32,-1679.034986,2,3
499997,"(a|DT, solo|JJ, piano|NN)",23,-1679.035464,3,2
499998,"(1962|CD, album|NN)",30,-1679.036008,2,6


In [7]:
min_freq = 20
eval_count = 500000

batch_count = max(ngram_eval.batch)

In [8]:
ngram_eval.batch.value_counts()

 1     170225
 2     129780
 3      77191
 4      53152
 5      30841
 6      17798
 7       9470
 8       5725
 9       2410
-1       1723
 10      1561
-2        124
Name: batch, dtype: int64

In [9]:
from nltk.tokenize import MWETokenizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Flatten down to a single number
def cosim(x,y):
    return cosine_similarity(x.reshape(1,-1), y.reshape(1,-1))[0][0]

In [10]:
def mwe_score(exp, model, stats_frame):
    
    # Combined token for MWE
    mwetoken = '+'.join(exp)

    # Stopwords - 1 if component is a stopword, 0 if present, -1 if simplex word missing from vocab, -2 if MWE missing
    sws = []
    # Component vectors
    cvs = []

    #  Neighbours in original & MWE-aware space
    #oldn = []
    #newn = []

    # List of individual word similarities (where present in the vocab)
    css = []

    # Empty array
    earr = np.empty(400)
    earr[:] = np.nan

    # Check that combined token exists in the vocab. This protects against inflation of n-gram counts caused by repeats
    #  of the same token (e.g. in lists like https://simple.wikipedia.org/wiki/List_of_cities,_towns_and_villages_in_Fars_Province)
    if mwetoken in batch_model.wv.vocab:

        mwv = model.wv[mwetoken]

        for w in exp:
            if w in model.wv.vocab:
                cvs.append(model.wv[w])

                #oldn.append(batch_model.wv.most_similar(w, topn=5))

                if w in stop:
                    sws.append(1)
                    css.append(np.nan)
                else:
                    sws.append(0)
                    css.append(cosim(model.wv[w], mwv ))

            # If component is absent from vocab
            else:
                sws.append(-1)
                cvs.append(earr)
                css.append(np.nan)

                #oldn.append([])

        #  Mean cosim
        if min(sws) >= 0:
            cs = np.nanmean(css)
        else:
            cs = np.nan

        #newn = batch_model.wv.most_similar(mwetoken, topn=5)

    # Combined token missing from vocab - mark with defaults
    else:
        sws = [-2]
        mwv = np.empty(400)
        mwv[:] = np.nan
        cs = np.nan


    # Append to stats df
    return stats_frame.append({
        'ngram'  : exp,
        'stopwords' : sws,
        'mwe_vector' : mwv,
        'component_vectors' : cvs,
        'component_cosims'  : css,
        'cosine_sim'  : cs,
        #'base_nearest': oldn,
        #'mwe_nearest' : newn,
    }, ignore_index=True)

In [11]:
%%time

batch_dfs = {}

for bb in range(batch_count):
    print('Processing batch {} of {}'.format(bb+1,batch_count))
    
    # Subset DataFrame
    batch_dfs[bb] = ngram_eval[ngram_eval.batch == bb+1].reset_index(drop=True)
    
    # Initialise MWETokenizer
    batch_token_mwe = MWETokenizer(list(batch_dfs[bb].ngram) , separator='+')
    
    # Load model
    print('Loading w2v model')

    batch_model = Word2Vec.load(datapath+'/Models/1 w2v/Tagged/w10p_tagged_batch{}.model'.format(bb+1))
    
    print('Gathering MWE stats')
    
    # For each MWE, evaluate stats. Record vectors (in case we want to calculate different metrics later).
    statsf = pd.DataFrame(columns=['ngram', 'stopwords', 'mwe_vector', 'component_vectors', 'component_cosims', 
                                   'cosine_sim']) # , 'base_nearest', 'mwe_nearest'
    batch_len = len(batch_dfs[bb].ngram)
    if batch_len >= 20000: 
        printer = 5000
    elif batch_len >= 5000: 
        printer = 1000
    else:
        printer = 200
        
    _i = 0
    
    for exp in batch_dfs[bb].ngram:
        if _i % printer == 0:
            print(' MWE '+str(_i)+'/'+str(batch_len)+': '+'+'.join(exp))
        _i += 1
        
        statsf = mwe_score(exp, batch_model, statsf)
        
    #  Join back onto DataFrame
    batch_dfs[bb] = batch_dfs[bb].merge(statsf, on='ngram')
    

Processing batch 1 of 10
Loading w2v model
Gathering MWE stats
 MWE 0/170225: 311.22|CD+500.86|CD
 MWE 5000/170225: prescription|NN+medication|NN
 MWE 10000/170225: Isao|NNP+Aoki|NNP
 MWE 15000/170225: Leydig|NN+cells|NNS
 MWE 20000/170225: Bijou|NNP+Theatre|NNP
 MWE 25000/170225: Zafar|NNP+Iqbal|NNP
 MWE 30000/170225: Forest|NNP+Library|NNP
 MWE 35000/170225: John|NNP+Michell|NNP
 MWE 40000/170225: of|IN+Mohandas|NNP
 MWE 45000/170225: and|CC+mixes|VBZ
 MWE 50000/170225: been|VBN+riding|VBG
 MWE 55000/170225: investigate|VB+their|PRP$
 MWE 60000/170225: magic|NN+wand|NN
 MWE 65000/170225: and|CC+Meng|NNP
 MWE 70000/170225: division|NN+received|VBD
 MWE 75000/170225: till|IN+9|CD
 MWE 80000/170225: French|JJ+elections|NNS
 MWE 85000/170225: BB|NNP+1|CD
 MWE 90000/170225: major|JJ+revisions|NNS
 MWE 95000/170225: would|MD+suffice|VB
 MWE 100000/170225: Paulo|NNP+Brazil|NNP+Clay|NNP
 MWE 105000/170225: own|JJ+relationship|NN
 MWE 110000/170225: in|IN+sunny|JJ
 MWE 115000/170225: Mac|NNP+

In [12]:
# Merge dataframes, sort by compositionality metric, export

# Also want the default batches with batch no < 0
all_batches = ngram_eval[ngram_eval.batch < 0]

for d in range(batch_count):
    print('Appending batch {} of '.format(d+1)+str(batch_count))
    all_batches = all_batches.append(batch_dfs[d])
    
all_batches = all_batches.sort_values('cosine_sim')
all_batches = all_batches.reset_index(drop=True)

all_batches.to_csv(datapath+'/Models/1 w2v/Tagged/Results/w10p_tagged_output_001.csv', index=False)

Appending batch 1 of 10
Appending batch 2 of 10
Appending batch 3 of 10
Appending batch 4 of 10
Appending batch 5 of 10
Appending batch 6 of 10
Appending batch 7 of 10
Appending batch 8 of 10
Appending batch 9 of 10
Appending batch 10 of 10


In [13]:
all_batches

,ngram,freq,poisson,len,batch,stopwords,mwe_vector,component_vectors,component_cosims,cosine_sim
0,"(delivered|VBD, from|IN)",28,-1567.428918,2,5,"[0, 1]","[-0.025481936, 0.0074053947, 0.092315055, -0.1...","[[-0.77035004, -0.2205575, -1.4344151, 2.03640...","[-0.2535712, nan]",-0.253571
1,"(then|RB, had|VBD, two|CD)",20,-1586.812073,3,3,"[0, 1, 1]","[-0.0629135, -0.06837138, 0.085801765, 0.06089...","[[-0.95267797, -0.12056105, -1.2403282, 0.5347...","[-0.24625674, nan, nan]",-0.246257
2,"(and|CC, finally|RB, for|IN)",20,-1647.756550,3,4,"[1, 0, 1]","[0.055126164, 0.026781093, -0.00090568303, -0....","[[-0.79431534, 0.19128917, -0.38475975, 0.3156...","[nan, -0.22293654, nan]",-0.222937
3,"(one|CD, before|IN, it|PRP)",20,-1587.727730,3,7,"[1, 0, 1]","[-0.15004537, 0.029226098, 0.09330798, 0.00089...","[[-0.97791326, -0.12678225, 0.62892336, 1.2675...","[nan, -0.21940775, nan]",-0.219408
4,"(However|RB, in|IN, their|PRP$)",20,-1642.620348,3,3,"[0, 1, 1]","[-0.075696215, 0.04566814, 0.058888163, -0.015...","[[-0.6142879, -0.22286342, -1.1887616, -1.8442...","[-0.2108197, nan, nan]",-0.210820
...,...,...,...,...,...,...,...,...,...,...
499995,"(the|DT, Barebones|NNPS, Parliament|NNP)",23,-1552.893367,3,10,"[1, -1, 0]","[-0.11529744, -0.1222, -0.048948135, -0.115126...","[[-0.25977308, 0.46199447, 0.31271642, 0.31649...","[nan, nan, 0.38508964]",NaN
499996,"(2nd|JJ, 2nd|JJ, 2nd|JJ)",24,-1598.434969,3,10,[-2],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",[],[],NaN
499997,"(was|VBD, 49,445|CD, in|IN)",22,-1600.048879,3,10,"[1, -1, 1]","[-0.14426154, -0.18483189, 0.08281145, -0.1432...","[[1.4046279, -0.057345472, 0.6591206, -1.05324...","[nan, nan, nan]",NaN
499998,"(a|DT, modicum|NN, of|IN)",22,-1637.924770,3,10,"[1, -1, 1]","[-0.1328804, 0.028195623, 0.016251683, -0.0018...","[[0.40583652, 0.099237256, -0.2926421, 0.45965...","[nan, nan, nan]",NaN


In [ ]:
#type(all_batches.stopwords[0])

In [14]:
# RuntimeWarning: Mean of empty slice
#  Appear to be taking the np.nanmean() of a list of NaNs

def inval(inlist,val=0):
    if type(inlist) != list:
        return False
    return val in inlist

#nozero = all_batches[~all_batches.stopwords.apply(inval)]

#minone = nozero[nozero.stopwords.apply(inval,val=-1)]

#minone[minone.stopwords.apply(inval,val=1)]

In [15]:
w10p_res_light = all_batches.drop(columns=['mwe_vector', 'component_vectors'])

In [17]:
w10p_res_light.to_csv(datapath+'/Models/1 w2v/Tagged/Results/w10p_tagged_light_001.csv', index=False)